# This notebook compares the CN map generated using the CN calculation with a product in google earth engine
##### Author: Omid Emamjomehzadeh (https://www.omidemam.com/)
##### Supervisor: Dr. Omar Wani (https://engineering.nyu.edu/faculty/omar-wani)
##### Hydrologic Systems Group @NYU (https://www.omarwani.com/)

In [1]:
# load libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
from matplotlib import rcParams
import re
import os
import glob
from tqdm import tqdm
import datetime

In [2]:
import warnings
# Suppress the specific RuntimeWarning
warnings.filterwarnings("ignore")
# Suppress all UserWarnings, including PerformanceWarnings
warnings.simplefilter('ignore', category=UserWarning)
# plot with Arial font
plt.rcParams['font.family'] = 'Arial'

#### this notebook compares the lumped CN value derived from the CN map computed using the CN map calculation notebook with the CN value derived from using the GEE product

In [3]:
path=r'D:\hydrological analysis\data\Peak hydrologic discharge\culvert_CN_tc_S_Ia.xlsx'
culvert = pd.read_excel(path)


In [4]:
culvert[['BIN','CN','CN_dry','CN_wet']].head()

,BIN,CN,CN_dry,CN_wet
0,C810530,77.519808,60.601441,88.865546
1,C810550,74.375262,56.567086,86.932914
2,C810490,68.429505,50.051354,82.789916
3,C810520,76.450000,58.788095,88.459524
4,C810491,69.028981,50.702707,83.089013


In [5]:
# Define the root directory
root_dir = r'D:\culvert repo\Results\waterhsed_delineation_dir'
# Define the pattern to search for shapefiles
pattern = os.path.join(root_dir, '**', '*watershed_poly_*m.shp')
# Use glob to find all matching shapefiles
shapefile_paths = glob.glob(pattern, recursive=True)
# Print the number of shapefiles found
print(f"Found {len(shapefile_paths)} shapefiles.")

Found 29960 shapefiles.


In [6]:
def select_closest_area(row):
    error = str(row['Closest_Area'])
    return error

In [7]:
CN_path = r"D:\culvert repo\data\initial layers\GEE\GCN250_Average_NY_reprojected.tif"
# Define the root directory to search in
# Progress bar
for idx, row in tqdm(culvert.iterrows(), total=culvert.shape[0], desc='number of processed rows'):
    error=select_closest_area(row)
    ###############################################################################################################################################
    # Find the path to the file 
    # Construct the specific filename pattern
    specific_pattern = f"{culvert['BIN'].iloc[idx]}watershed_poly_{error}m.shp"
    # Find the path related to the specific pattern
    matching_path = None
    for path in shapefile_paths:
        if specific_pattern in os.path.basename(path):
            matching_path = path
            break
    if matching_path is None:
        print("No matching shapefile found.")
        culvert.loc[idx, f"CN"] = np.nan
        continue
    ###############################################################################################################################################
    watershed_polygone = matching_path
    watershed = gpd.read_file(watershed_polygone)
    # Convert the watershed boundary to GeoJSON-like format
    watershed_geom = [feature["geometry"] for feature in watershed.__geo_interface__["features"]]
    ###############################################################################################################################################
    # Open the precipitation TIFF file and clip it using the watershed boundary
    with rasterio.open(CN_path) as src:
        out_image, out_transform = mask(src, watershed_geom, crop=True, all_touched=True)
        # Set NoData value
        out_image[out_image == src.nodata] = 0
        # Calculate the mean of the clipped raster, excluding NoData values
        if np.count_nonzero(out_image) == 0:  # Check if the data is empty (no non-zero values)
            mean_value = np.nan
        else:
            mean_value = np.nanmean(np.where(out_image == 0, np.nan, out_image))
    # Update the DataFrame with the mean value for the given error
    culvert.loc[idx, f"GEE_CN"] = mean_value

number of processed rows: 100%|██████████| 2418/2418 [03:57<00:00, 10.17it/s]


In [8]:
CN_path = r"D:\culvert repo\data\initial layers\GEE\GCN250_Wet_NY_reprojected.tif"
# Define the root directory to search in
# Progress bar
for idx, row in tqdm(culvert.iterrows(), total=culvert.shape[0], desc='number of processed rows'):
    error=select_closest_area(row)
    ###############################################################################################################################################
    # Find the path to the file 
    # Construct the specific filename pattern
    specific_pattern = f"{culvert['BIN'].iloc[idx]}watershed_poly_{error}m.shp"
    # Find the path related to the specific pattern
    matching_path = None
    for path in shapefile_paths:
        if specific_pattern in os.path.basename(path):
            matching_path = path
            break
    if matching_path is None:
        print("No matching shapefile found.")
        culvert.loc[idx, f"CN"] = np.nan
        continue
    ###############################################################################################################################################
    watershed_polygone = matching_path
    watershed = gpd.read_file(watershed_polygone)
    # Convert the watershed boundary to GeoJSON-like format
    watershed_geom = [feature["geometry"] for feature in watershed.__geo_interface__["features"]]
    ###############################################################################################################################################
    # Open the precipitation TIFF file and clip it using the watershed boundary
    with rasterio.open(CN_path) as src:
        out_image, out_transform = mask(src, watershed_geom, crop=True, all_touched=True)
        # Set NoData value
        out_image[out_image == src.nodata] = 0
        # Calculate the mean of the clipped raster, excluding NoData values
        if np.count_nonzero(out_image) == 0:  # Check if the data is empty (no non-zero values)
            mean_value = np.nan
        else:
            mean_value = np.nanmean(np.where(out_image == 0, np.nan, out_image))
    # Update the DataFrame with the mean value for the given error
    culvert.loc[idx, f"GEE_CN_wet"] = mean_value

number of processed rows: 100%|██████████| 2418/2418 [02:08<00:00, 18.75it/s]


In [9]:
CN_path = r"D:\culvert repo\data\initial layers\GEE\GCN250_Dry_NY_reprojected.tif"
# Define the root directory to search in
# Progress bar
for idx, row in tqdm(culvert.iterrows(), total=culvert.shape[0], desc='number of processed rows'):
    error=select_closest_area(row)
    ###############################################################################################################################################
    # Find the path to the file 
    # Construct the specific filename pattern
    specific_pattern = f"{culvert['BIN'].iloc[idx]}watershed_poly_{error}m.shp"
    # Find the path related to the specific pattern
    matching_path = None
    for path in shapefile_paths:
        if specific_pattern in os.path.basename(path):
            matching_path = path
            break
    if matching_path is None:
        print("No matching shapefile found.")
        culvert.loc[idx, f"CN"] = np.nan
        continue
    ###############################################################################################################################################
    watershed_polygone = matching_path
    watershed = gpd.read_file(watershed_polygone)
    # Convert the watershed boundary to GeoJSON-like format
    watershed_geom = [feature["geometry"] for feature in watershed.__geo_interface__["features"]]
    ###############################################################################################################################################
    # Open the precipitation TIFF file and clip it using the watershed boundary
    with rasterio.open(CN_path) as src:
        out_image, out_transform = mask(src, watershed_geom, crop=True, all_touched=True)
        # Set NoData value
        out_image[out_image == src.nodata] = 0
        # Calculate the mean of the clipped raster, excluding NoData values
        if np.count_nonzero(out_image) == 0:  # Check if the data is empty (no non-zero values)
            mean_value = np.nan
        else:
            mean_value = np.nanmean(np.where(out_image == 0, np.nan, out_image))
    # Update the DataFrame with the mean value for the given error
    culvert.loc[idx, f"GEE_CN_dry"] = mean_value

number of processed rows: 100%|██████████| 2418/2418 [02:09<00:00, 18.69it/s]


In [10]:
culvert[['BIN','CN','CN_dry','CN_wet','GEE_CN','GEE_CN_dry','GEE_CN_wet']].head()

,BIN,CN,CN_dry,CN_wet,GEE_CN,GEE_CN_dry,GEE_CN_wet
0,C810530,77.519808,60.601441,88.865546,71.750000,52.730769,85.826923
1,C810550,74.375262,56.567086,86.932914,70.023529,50.917647,84.623529
2,C810490,68.429505,50.051354,82.789916,73.550000,55.150000,87.025000
3,C810520,76.450000,58.788095,88.459524,72.000000,53.000000,86.000000
4,C810491,69.028981,50.702707,83.089013,72.052326,53.279070,86.023256


### Average absolute difference

In [12]:
error_pairs = [
    ('CN', 'GEE_CN'),
    ('CN_dry', 'GEE_CN_dry'),
    ('CN_wet', 'GEE_CN_wet'),
]

for a, b in error_pairs:
    errors = culvert[a] - culvert[b]
    mae = errors.abs().mean()
    std = errors.std()
    print(f"{a} vs {b}: MAE = {mae:.2f}, STD = {std:.2f}")

CN vs GEE_CN: MAE = 4.73, STD = 5.46
CN_dry vs GEE_CN_dry: MAE = 6.27, STD = 6.44
CN_wet vs GEE_CN_wet: MAE = 2.71, STD = 3.63


In [34]:
# Date and time
now = datetime.datetime.now()
print(f"Date and time: {now}")

Date and time: 2025-05-01 16:23:55.882762


In [35]:
%load_ext watermark
# Print the Python version and some dependencies
%watermark -v -m -p numpy,pandas,geopandas,matplotlib,os,rasterio,tqdm

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.20.0

numpy     : 2.0.2
pandas    : 2.2.2
geopandas : 1.0.1
matplotlib: 3.8.4
os        : unknown
rasterio  : 1.4.3
tqdm      : 4.66.5

Compiler    : MSC v.1940 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 183 Stepping 1, GenuineIntel
CPU cores   : 24
Architecture: 64bit

